# 开始使用 Captum

[Captum](https://captum.ai/)（拉丁语中意为“理解”）是一个基于 PyTorch 的开源可扩展模型可解释性库。

随着模型复杂性的增加以及由此导致的透明性缺乏，模型可解释性方法变得越来越重要。模型理解既是一个活跃的研究领域，也是使用机器学习的各行业实际应用的重点领域。Captum 提供了包括 Integrated Gradients 在内的最先进算法，为研究人员和开发人员提供了一种简单的方法来理解哪些特征对模型输出有贡献。

完整文档、API 参考以及一套关于特定主题的教程可在 [captum.ai](https://captum.ai/) 网站上找到。

## 简介

Captum 的模型可解释性方法是基于*归因*的。Captum 提供三种归因类型：

* **特征归因**：试图通过生成输出的输入特征来解释特定输出。例如，通过某些评论中的词语来解释电影评论是正面还是负面。
* **层归因**：检查模型隐藏层在特定输入后的活动。例如，检查卷积层对输入图像的空间映射输出。
* **神经元归因**：类似于层归因，但专注于单个神经元的活动。

在这个交互式笔记本中，我们将研究特征归因和层归因。

每种归因类型都有多个**归因算法**。许多归因算法分为两大类：

* **基于梯度的算法**：计算模型输出、层输出或神经元激活相对于输入的反向梯度。例如，**Integrated Gradients**（特征）、**Layer Gradient * Activation** 和 **Neuron Conductance**。
* **基于扰动的算法**：通过改变输入来检查模型、层或神经元输出的变化。例如，**Occlusion**、**Feature Ablation** 和 **Feature Permutation**。

我们将在下面研究这两种类型的算法。

尤其是在涉及大型模型时，将归因数据可视化为与被检查输入特征相关的方式可能非常有价值。虽然可以使用 Matplotlib、Plotly 或类似工具创建自己的可视化，但 Captum 提供了专门针对其归因的增强工具：

* `captum.attr.visualization` 模块（以下导入为 `viz`）提供了有助于可视化与图像相关的归因的功能。
* **Captum Insights** 是 Captum 之上的一个易于使用的 API，提供了一个可视化小部件，具有针对图像、文本和任意模型类型的现成可视化。

这两种可视化工具将在本笔记本中演示。前几个示例将重点关注计算机视觉用例，但最后的 Captum Insights 部分将演示多模型视觉问答模型中的归因可视化。

## 安装

在开始之前，您需要一个 Python 环境，其中包括：

* Python 版本 3.6 或更高
* 对于 Captum Insights 示例，需要 Flask 1.1 或更高版本
* PyTorch 版本 1.2 或更高（推荐最新版本）
* TorchVision 版本 0.6 或更高（推荐最新版本）
* Captum（推荐最新版本）

在 Anaconda 或 pip 虚拟环境中安装 Captum，请使用以下适合您环境的命令：

```
conda install captum -c pytorch
pip install captum
```

在您设置的环境中重新启动此笔记本，即可开始！

## 第一个示例

首先，让我们从一个简单的视觉示例开始。我们将从一个在 ImageNet 数据集上预训练的 ResNet 模型开始。我们将获取一个测试输入，并使用不同的**特征归因**算法来检查输入图像如何影响输出，并查看一些测试图像的输入归因图的有用可视化。

首先，一些导入：

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models

import captum
from captum.attr import IntegratedGradients, Occlusion, LayerGradCam, LayerAttribution
from captum.attr import visualization as viz

import os, sys
import json

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# 导入必要的库，包括 PyTorch、TorchVision 和 Captum

现在我们将使用 TorchVision 模型库下载一个预训练的 ResNet。由于我们不进行训练，我们将暂时将其置于评估模式。

In [ ]:
model = models.resnet101(pretrained=True)
model = model.eval()

# 下载预训练的 ResNet 模型并设置为评估模式

您获取此交互式笔记本的地方应该还有一个 `img` 文件夹，其中有一个名为 `cat.jpg` 的文件。

In [ ]:
test_img = Image.open('img/cat.jpg')
test_img_data = np.asarray(test_img)
plt.imshow(test_img_data)
plt.show()

# 加载测试图像并显示

我们的 ResNet 模型是在 ImageNet 数据集上训练的，期望图像具有特定大小，并且通道数据归一化到特定范围的值。我们还将获取模型识别的类别的人类可读标签列表 - 这也应该在 `img` 文件夹中。

In [ ]:
# 模型期望 224x224 的三色图像
transform = transforms.Compose([
 transforms.Resize(224),
 transforms.CenterCrop(224),
 transforms.ToTensor()
])

# 标准 ImageNet 归一化
transform_normalize = transforms.Normalize(
     mean=[0.485, 0.456, 0.406],
     std=[0.229, 0.224, 0.225]
 )

transformed_img = transform(test_img)
input_img = transform_normalize(transformed_img)
input_img = input_img.unsqueeze(0) # 模型需要一个虚拟批次维度

labels_path = 'img/imagenet_class_index.json'
with open(labels_path) as json_data:
    idx_to_labels = json.load(json_data)

# 转换图像并加载标签

现在，我们可以问一个问题：我们的模型认为这张图像代表什么？

In [ ]:
output = model(input_img)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)
pred_label_idx.squeeze_()
predicted_label = idx_to_labels[str(pred_label_idx.item())][1]
print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')

# 获取模型预测的类别和置信度

我们已经确认 ResNet 认为我们的猫图像确实是一只猫。但*为什么*模型认为这是猫的图像呢？

为了解答这个问题，我们转向 Captum。

## 使用 Integrated Gradients 进行特征归因

**特征归因**将特定输出归因于输入的特征。它使用特定输入 - 在这里是我们的测试图像 - 生成一个相对重要性的输入特征图。

[**Integrated Gradients**](https://captum.ai/api/integrated_gradients.html) 是 Captum 提供的特征归因算法之一。Integrated Gradients 通过近似模型输出相对于输入的梯度积分，为每个输入特征分配重要性分数。

在我们的例子中，我们将选择输出向量的一个特定元素 - 即模型对其选择类别的置信度 - 并使用 Integrated Gradients 来理解输入图像的哪些部分对该输出有贡献。

一旦我们从 Integrated Gradients 获得重要性图，我们将使用 Captum 的可视化工具提供一个有用的表示。Captum 的 `visualize_image_attr()` 函数提供了多种选项来自定义归因数据的显示。在这里，我们传入一个自定义的 Matplotlib 颜色图。

运行包含 `integrated_gradients.attribute()` 调用的单元格通常需要一两分钟。

In [ ]:
# 使用模型初始化归因算法
integrated_gradients = IntegratedGradients(model)

# 请求算法将输出目标归因于输入
attributions_ig = integrated_gradients.attribute(input_img, target=pred_label_idx, n_steps=200)

# 使用 Integrated Gradients 计算归因

In [ ]:
# 显示原始图像以供比较
_ = viz.visualize_image_attr(None, np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)), 
                      method="original_image", title="Original Image")

default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                 [(0, '#ffffff'),
                                                  (0.25, '#0000ff'),
                                                  (1, '#0000ff')], N=256)

_ = viz.visualize_image_attr(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='positive',
                             title='Integrated Gradients')

# 可视化 Integrated Gradients 的归因结果

在上图中，您应该看到 Integrated Gradients 在图像中猫的位置周围给出了最强的信号。

## 使用 Occlusion 进行特征归因

基于梯度的归因方法通过直接计算输出相对于输入的变化来帮助理解模型。*基于扰动的归因*方法更直接，通过引入输入的变化来测量对输出的影响。[**Occlusion**](https://captum.ai/api/occlusion.html) 是一种这样的方法。它涉及替换输入图像的部分，并检查对输出信号的影响。

下面，我们设置 Occlusion 归因。类似于配置卷积神经网络，您可以指定目标区域的大小，以及步长以确定单个测量的间距。我们将使用 `visualize_image_attr_multiple()` 可视化 Occlusion 归因的输出，显示区域的正负归因热图，并通过遮蔽原始图像的正归因区域进行显示。遮蔽提供了一个非常有启发性的视图，显示了模型认为我们猫照片中最“像猫”的区域。

In [ ]:
occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(input_img,
                                       target=pred_label_idx,
                                       strides=(3, 8, 8),
                                       sliding_window_shapes=(3,15, 15),
                                       baselines=0)

# 使用 Occlusion 计算归因

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map", "heat_map", "masked_image"],
                                      ["all", "positive", "negative", "positive"],
                                      show_colorbar=True,
                                      titles=["Original", "Positive Attribution", "Negative Attribution", "Masked"],
                                      fig_size=(18, 6)
                                     )

# 可视化 Occlusion 的归因结果

再次，我们看到图像中包含猫的区域被赋予了更大的重要性。

## 使用 Layer GradCAM 进行层归因

**层归因**允许您将模型隐藏层的活动归因于输入的特征。下面，我们将使用一个层归因算法来检查模型中一个卷积层的活动。

GradCAM 计算目标输出相对于给定层的梯度，对每个输出通道（输出的维度 2）进行平均，并将每个通道的平均梯度乘以层激活。结果在所有通道上求和。GradCAM 专为卷积网络设计；由于卷积层的活动通常在空间上映射到输入，GradCAM 归因通常被上采样并用于遮蔽输入。

层归因的设置与输入归因类似，除了除了模型之外，您还必须指定要检查的模型隐藏层。与上面一样，当我们调用 `attribute()` 时，我们指定感兴趣的目标类别。

In [ ]:
layer_gradcam = LayerGradCam(model, model.layer3[1].conv2)
attributions_lgc = layer_gradcam.attribute(input_img, target=pred_label_idx)

# 使用 Layer GradCAM 计算层归因

In [ ]:
_ = viz.visualize_image_attr(attributions_lgc[0].cpu().permute(1,2,0).detach().numpy(),
                             sign="all",
                             title="Layer 3 Block 1 Conv 2")

# 可视化 Layer GradCAM 的归因结果

我们将使用 [**LayerAttribution**](https://captum.ai/api/base_classes.html?highlight=layerattribution#captum.attr.LayerAttribution) 基类中的便利方法 `interpolate()` 来上采样此归因数据，以便与输入图像进行比较。

In [ ]:
upsamp_attr_lgc = LayerAttribution.interpolate(attributions_lgc, input_img.shape[2:])

print(attributions_lgc.shape)
print(upsamp_attr_lgc.shape)
print(input_img.shape)

_ = viz.visualize_image_attr_multiple(upsamp_attr_lgc[0].cpu().permute(1,2,0).detach().numpy(),
                                      transformed_img.permute(1,2,0).numpy(),
                                      ["original_image","blended_heat_map","masked_image"],
                                      ["all","positive","positive"],
                                      show_colorbar=True,
                                      titles=["Original", "Positive Attribution", "Masked"],
                                      fig_size=(18, 6))

# 上采样 Layer GradCAM 的归因数据并可视化

这样的可视化可以让您对隐藏层如何响应输入有新的见解。

## 使用 Captum Insights 进行可视化

Captum Insights 是一个基于 Captum 构建的解释性可视化小部件，用于促进模型理解。Captum Insights 可跨图像、文本和其他特征工作，帮助用户理解特征归因。它允许您可视化多个输入/输出对的归因，并提供图像、文本和任意数据的可视化工具。

在本笔记本的这一部分中，我们将使用 Captum Insights 可视化多个图像分类推断。

首先，让我们收集一些图像，看看模型对它们的看法。为了多样性，我们将选择我们的猫、一个茶壶和一个三叶虫化石：

In [ ]:
imgs = ['img/cat.jpg', 'img/teapot.jpg', 'img/trilobite.jpg']

for img in imgs:
    img = Image.open(img)
    transformed_img = transform(img)
    input_img = transform_normalize(transformed_img)
    input_img = input_img.unsqueeze(0) # 模型需要一个虚拟批次维度

    output = model(input_img)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)
    pred_label_idx.squeeze_()
    predicted_label = idx_to_labels[str(pred_label_idx.item())][1]
    print('Predicted:', predicted_label, '/', pred_label_idx.item(), ' (', prediction_score.squeeze().item(), ')')

# 加载图像并获取模型预测

...看起来我们的模型正在正确识别它们 - 但当然，我们想要深入挖掘。为此，我们将使用 Captum Insights 小部件，我们通过 `AttributionVisualizer` 对象进行配置，下面导入。`AttributionVisualizer` 期望数据批次，因此我们将引入 Captum 的 `Batch` 帮助类。我们将专门查看图像，因此还将导入 `ImageFeature`。

我们使用以下参数配置 `AttributionVisualizer`：

* 要检查的模型数组（在我们的例子中，只有一个）
* 评分函数，允许 Captum Insights 从模型中提取前 k 个预测
* 我们模型训练的类别的有序、人类可读列表
* 要查找的特征列表 - 在我们的例子中，是一个 `ImageFeature`
* 数据集，这是一个返回输入和标签批次的可迭代对象 - 就像您用于训练一样

In [ ]:
from captum.insights import AttributionVisualizer, Batch
from captum.insights.attr_vis.features import ImageFeature

# 基线是全零输入 - 这可能因您的数据而异
def baseline_func(input):
    return input * 0

# 合并我们上面的图像转换
def full_img_transform(input):
    i = Image.open(input)
    i = transform(i)
    i = transform_normalize(i)
    i = i.unsqueeze(0)
    return i


input_imgs = torch.cat(list(map(lambda i: full_img_transform(i), imgs)), 0)

visualizer = AttributionVisualizer(
    models=[model],
    score_func=lambda o: torch.nn.functional.softmax(o, 1),
    classes=list(map(lambda k: idx_to_labels[k][1], idx_to_labels.keys())),
    features=[
        ImageFeature(
            "Photo",
            baseline_transforms=[baseline_func],
            input_transforms=[],
        )
    ],
    dataset=[Batch(input_imgs, labels=[282,849,69])]
)

# 配置 Captum Insights 的 AttributionVisualizer

注意运行上面的单元格并没有花费太多时间，不像我们上面的归因。这是因为 Captum Insights 允许您在可视化小部件中配置不同的归因算法，然后它将计算并显示归因。*这个*过程将花费几分钟。

运行下面的单元格将呈现 Captum Insights 小部件。然后，您可以选择归因方法及其参数，基于预测类别或预测正确性过滤模型响应，查看模型的预测及其关联概率，并查看归因的热图与原始图像的比较。

In [ ]:
visualizer.render()

# 渲染 Captum Insights 小部件

In [ ]:

# 结束代码块